In [ ]:
import pandas as pd
import numpy as np
from my_package import userinput, filemanager, ospath

pd.set_option('display.max_colwidth', None)
pd.set_option("mode.copy_on_write", True) # see: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#copy-on-write-chained-assignment

# Create a collection
collection_name = "products_collection"

input_dir = userinput.get_user_input("Input directory", default=ospath.get_download_folder())
print(f"Indexing data tables from directory: {input_dir}")
file_list = filemanager.get_files_in_directory(input_dir, extensions=[".xlsx", ".xls"])
filemanager.print_enumerated_file_list(file_list)
selected_file = userinput.get_user_input("Select a file by number", default="1")
file_path = file_list[int(selected_file) - 1]
# print(f"Loading data from: {file_path}")

def get_nice_strings(lst):
    return [lst.strip().replace('\n ', '').replace('\n', '').replace(' ', '_').lower() for lst in lst]


def get_nice_string(s):
    return s.strip().replace('\n ', '').replace('\n', '').replace(' ', '_').lower()


def get_duplicate_column_names(dataframe, level=0):
    # declaring a set for duplicated column names
    dupcols_set = set()

    # collecting column names in given level
    colnames = dataframe.columns.get_level_values(level)

    return list({x for x in colnames if x in dupcols_set or dupcols_set.add(x)})


def get_unique_column_names(dataframe, level=0):
    cols = dataframe.columns.get_level_values(level)
    return list(set(cols))


def get_reshaped_dataframe(file_path='../input/20260102-MFI-Products-anonymized.xlsx'):
    df_i = pd.read_excel(file_path, header=[0,1]) if file_path.lower().endswith(('.xlsx', '.xls')) else pd.read_csv(file_path, encoding='utf_8', sep=';')

    dupcols = get_duplicate_column_names(df_i, level=0)
    unique_cols = get_unique_column_names(df_i, level=0)

    for col in unique_cols:

        new_colname = get_nice_string(col)

        if (col in dupcols):
            df_temp = df_i.xs(col, level=0, axis=1, drop_level=False)
            colnames = df_temp.columns.get_level_values(1)
            new_colnames = get_nice_strings(colnames)
            df_temp.columns = new_colnames[:]
            df_temp.replace('x', pd.Series(df_temp.columns, df_temp.columns), inplace=True)
            df_temp[new_colname] = [v[pd.notna(v)] for v in df_temp.values]
            df[new_colname] = df_temp.loc[:, new_colname]
        else:
            df[new_colname] = df_i.loc[:, col]

    cols = list(df)
    cols.insert(0, cols.pop(cols.index('product_name_alias')))
    df = df.loc[:, cols]
    return df


df = pd.DataFrame()
df = get_reshaped_dataframe(file_path)
df.head(-10)

Indexing data tables from directory: /Users/andreas/Downloads
1. /Users/andreas/Downloads/20251002-MEGGLE-Food-Ingredients-Products.xlsx
2. /Users/andreas/Downloads/20251230-MFI-Products-cleaned.xlsx
3. /Users/andreas/Downloads/20251002-Produktfinder-überarbeitet.xlsx
4. /Users/andreas/Downloads/url_redirects_slug_match.xlsx
5. /Users/andreas/Downloads/redirect-list-gemini-3.xlsx
6. /Users/andreas/Downloads/20250916-MEGGLE-Angebote_vs_Iststand.xlsx
7. /Users/andreas/Downloads/20250902-MEGGLE-Fragen-zur-Stundenerfassung.xlsx
8. /Users/andreas/Downloads/Vergleich_URLs.xlsx
9. /Users/andreas/Downloads/20260102-MFI-Products-anonymized.xlsx
10. /Users/andreas/Downloads/Strukturbaum.xlsx
11. /Users/andreas/Downloads/20251007-MEGGLE-Foodservice-Kalkulation-Nachtrag.xlsx


/Users/andreas/Code/powerspots/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


UnboundLocalError: cannot access local variable 'df' where it is not associated with a value

In [415]:
def get_duplicate_column_names(dataframe, level=0):
    # declaring a set for duplicated column names
    dupcols_set = set()

    # collecting column names in given level
    colnames = dataframe.columns.get_level_values(level)

    return list({x for x in colnames if x in dupcols_set or dupcols_set.add(x)})


def get_unique_column_names(dataframe, level=0):
    cols = df.columns.get_level_values(level)
    return list(set(cols))

dupcols = get_duplicate_column_names(df, level=0)
unique_cols = get_unique_column_names(df, level=0)

print(dupcols)
print(unique_cols)

['Product segment', 'Certifications', 'Dietary Forms', 'Fat source', 'Fat type', 'Protein content', 'Common Applications  ', 'Product Features', 'Fat content']
['Product Group  ', 'Product Family ', 'Product segment', 'Certifications', 'product_name_alias', 'Dietary Forms', 'Fat source', 'MEGGLE´s corresponding Product Family', 'Fat type', 'Protein content', 'Common Applications  ', 'Product Features', 'Fat content']


In [ ]:
def get_nice_strings(lst):
    return [lst.strip().replace('\n ', '').replace('\n', '').replace(' ', '_').lower() for lst in lst]

def get_nice_string(s):
    return s.strip().replace('\n ', '').replace('\n', '').replace(' ', '_').lower()

foo = pd.DataFrame()

def get_reshaped_dataframe(file_path):

for col in unique_cols:

    new_colname = get_nice_string(col)

    if (col in dupcols):
        df_temp = df.xs(col, level=0, axis=1, drop_level=False)
        colnames = df_temp.columns.get_level_values(1)
        new_colnames = get_nice_strings(colnames)
        df_temp.columns = new_colnames[:]
        df_temp.replace('x', pd.Series(df_temp.columns, df_temp.columns), inplace=True)
        df_temp[new_colname] = [v[pd.notna(v)] for v in df_temp.values]
        foo[new_colname] = df_temp.loc[:, new_colname]
    else:
        foo[new_colname] = df.loc[:, col]

cols = list(foo)
cols.insert(0, cols.pop(cols.index('product_name_alias')))
foo = foo.loc[:, cols]
foo.head(-10)

,product_name_alias,product_group,product_family,product_segment,certifications,dietary_forms,fat_source,meggle´s_corresponding_product_family,fat_type,protein_content,common_applications,product_features,fat_content
0,Product 1,Functional Compounds,Functional Milk Protein Compounds,[bakery_&_sweets],"[kosher, halal]","[vegetarian, gluten_free]",[],Burolac®,[],[>_80],[egg_substitution_and_egg_reduction],[clean_label],[]
1,Product 2,Functional Compounds,Functional Milk Protein Compounds,[bakery_&_sweets],"[kosher, halal]","[vegetarian, gluten_free]",[],Burolac®,[],[>_80],[egg_substitution_and_egg_reduction],[clean_label],[]
2,Product 3,Milk & Whey Powder,Buttermilkpowder,[],"[kosher, halal]","[vegetarian, gluten_free]",[],Buttermilkpowder,[],[],[],[clean_label],[]
3,Product 4,Functional Compounds,Functional Milk Protein Compounds,[bakery_&_sweets],"[kosher, halal]","[vegetarian, gluten_free]",[],Casinella®,[],[>_80],[],[clean_label],[]
4,Product 5,Functional Compounds,Functional Milk Protein Compounds,"[bakery_&_sweets, single_ingredients]","[kosher, halal]","[vegetarian, gluten_free]",[],Casinella®,[],[>_80],[egg_substitution_and_egg_reduction],[clean_label],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Product 98,Functional Compounds,Stabilising Compounds for Dairy Products,[dairy],"[kosher, halal]","[vegetarian, gluten_free]",[],MTM®,[],[>_80],[yoghurt_&_sour_milk_products],[clean_label],[]
98,Product 99,Functional Compounds,Stabilising Compounds for Dairy Products,"[dairy, nutrition]","[kosher, halal]","[vegetarian, gluten_free]",[],MTM®,[],[50-80],"[yoghurt_&_sour_milk_products, protein_enriched_dairy_products]",[clean_label],[]
99,Product 100,Functional Compounds,Stabilising Compounds for Dairy Products,"[dairy, nutrition]","[kosher, halal]","[vegetarian, gluten_free]",[],MTM®,[],[>_80],"[yoghurt_&_sour_milk_products, protein_enriched_dairy_products]",[clean_label],[]
100,Product 101,Functional Compounds,Stabilising Compounds for Dairy Products,[dairy],"[kosher, halal]","[vegetarian, gluten_free]",[],MTM®,[],[>_80],[yoghurt_&_sour_milk_products],[clean_label],[]


In [ ]:
# Load the CSV file into a DataFrame
# the default file encoding type is 'utf_8', change to e.g. 'cp1252' if needed
# the default field separator is ';', change to something else (e.g. ',') if needed
df = filemanager.read_csv_file(selected_file)

drop_columns = ['shelflife', 'CN_code', 'country_of_origin']

df_temp = df.copy()
df_temp = df_temp.drop(columns=drop_columns)
df_temp = df_temp.rename(columns={'product_name_alias': 'title', 'product_description': 'description'})


# Function to keep only string values and True boolean values
def keep_string_or_true(row):
    kept = {}
    for col, val in row.items():
        if pd.isna(val):
            continue
        if isinstance(val, str):
            if val.strip() != "":
                kept[col] = val
        elif val is True:
            kept[col] = val
    return json.dumps(kept)

# Add a column with per-row dict of kept values
df_temp['json_data'] = df_temp.apply(keep_string_or_true, axis=1)


# Option A: keep the dicts (one cell per row)
# filtered_series = df_temp['filtered']

# Option B: expand dicts back to columns (sparse; missing entries become NaN)
# filtered_df = pd.json_normalize(filtered_series).reindex(df_temp.index)

# If you want df_temp to be the expanded result uncomment:
# df_temp = filtered_df


keep_columns = ['title', 'description', 'json_data']
df_temp = df_temp[keep_columns]



df_temp['embedding'] = df_temp['json_data'].apply(lambda x: generate_embeddings(x, "mxbai-embed-large"))
# df_temp.head(-3)

measurer = np.vectorize(len)

vector_size = measurer(df_temp['embedding']).max(axis=0)
# print(f"Embedding vector size: {vector_size}")
# print(f"Embedding vector sizes: min={vector_size.min()}, max={vector_size.max()}, mean={vector_size.mean()}")